In [49]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import requests
import statsmodels.formula.api as smf
import re


In [2]:
def load_data(file_path, url_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
    else:
        #process request in batches...
        d,i,offset_size = [],0,100000
        k = offset_size
        while k == offset_size:
            if i > 10:
                break
            r=requests.get(
                url = url_path, 
                params={
                    '$limit':offset_size, 
                    '$offset':offset_size*i
                }
            )
            as_df = pd.DataFrame(r.json())
            k = len(as_df)
            i += 1
            d.append(as_df)
    
        df = pd.concat(d)
        df.to_csv(file_path)
        df.head(10)
    return df

In [3]:
tree_census_df =load_data('tree_census/flatfiles/tree_census_15.csv', 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json')

In [4]:
air_quality_df = load_data('data/Air_Quality_20241206.csv', 'https://data.cityofnewyork.us/resource/c3uy-2p5r.json')

In [5]:
green_streets_df = load_data('data/Greenstreets.csv', 'https://data.cityofnewyork.us/resource/mk9u-qu7i.json')

In [6]:
property_valuation_df = load_data('data/Property_Valuation_and_Assessment_Data_20241125.csv', 'https://data.cityofnewyork.us/resource/yjxr-fw8i.json')

# Tree Census Data Sanitation 

In [7]:
cols = [
    'tree_id','block_id','the_geom','zipcode','cb_num','borocode','nta_name',
    'health','status','tree_dbh','spc_common','spc_latin','problems','sidewalk',
]

tree_census_tr = tree_census_df[cols]

#only consider alive trees
tree_census_tr = tree_census_tr.loc[tree_census_df['status']=='Alive']

#drop trees without a health label 
tree_census_tr = tree_census_tr.loc[~tree_census_tr['health'].isna()]
tree_census_tr['tree_dbh'] = tree_census_tr['tree_dbh'].astype(int)

tree_census_tr.head()

,tree_id,block_id,the_geom,zipcode,cb_num,borocode,nta_name,health,status,tree_dbh,spc_common,spc_latin,problems,sidewalk
0,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",11375,406,4,Forest Hills,Fair,Alive,3,red maple,Acer rubrum,NaN,NoDamage
1,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",11357,407,4,Whitestone,Fair,Alive,21,pin oak,Quercus palustris,Stones,Damage
2,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",11211,301,3,East Williamsburg,Good,Alive,3,honeylocust,Gleditsia triacanthos var. inermis,NaN,Damage
3,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",11211,301,3,East Williamsburg,Good,Alive,10,honeylocust,Gleditsia triacanthos var. inermis,Stones,Damage
4,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",11215,306,3,Park Slope-Gowanus,Good,Alive,21,American linden,Tilia americana,Stones,Damage


In [12]:
#make analysis at zipcode level...
cb_grp = tree_census_tr.groupby(['cb_num']).agg(
    tree_count=('tree_id', len), 
    tree_size=('tree_dbh', np.sum)
)

#avg tree size (diameter)
cb_grp['tree_size'] = cb_grp['tree_size']/cb_grp['tree_count']

#count trees per zip/health category (good, fair, bad), then pivot on the health cats
cb_grp_health_status = tree_census_tr.groupby(['cb_num','health'])[['tree_id']].count().reset_index()
cb_grp_health_status_piv = cb_grp_health_status.pivot(
    index='cb_num',
    columns='health', 
    values='tree_id'
)

#concat cb features
cb_grp_health_status_piv = cb_grp_health_status_piv.div(cb_grp['tree_count'], axis=0)
tree_cb_features = pd.concat((cb_grp_health_status_piv,cb_grp),axis=1)

tree_cb_features['Good'] = tree_cb_features['Good'].fillna(0.0)
tree_cb_features = tree_cb_features.rename(
    columns={
        'Good':'Tree Quality'
    }
)

tree_cb_features = tree_cb_features.reset_index()
tree_cb_features['cb_num'] = tree_cb_features['cb_num'].astype(str)
tree_cb_features = tree_cb_features[['cb_num', 'Tree Quality', 'tree_count']]
tree_cb_features.head()

/var/folders/wx/wz1pbs1903j_cfgqws400dz80000gn/T/ipykernel_70401/1516815439.py:2: FutureWarning: The provided callable <function sum at 0x11859a8b0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cb_grp = tree_census_tr.groupby(['cb_num']).agg(


,cb_num,Tree Quality,tree_count
0,101,0.764040,2297
1,102,0.730602,4833
2,103,0.764069,4709
3,104,0.824395,4419
4,105,0.673560,2031


In [105]:
cb_geometries = gpd.read_file("tree_census/flatfiles/Community Districts.geojson")
geo_cols = ['boro_cd','shape_area','geometry']
cb_geometries = cb_geometries[geo_cols].rename(
    columns={
        'boro_cd':'cb_num'
    }
)
tree_census_geoms = tree_cb_features.merge(
    cb_geometries, 
    how='inner', 
    on='cb_num'
)

#compute acreage over the cb district, calculate trees per acre
tree_census_geoms['cb_acreage'] = tree_census_geoms['shape_area'].astype(np.float64)/43560
tree_census_geoms['Tree Density (per Acre)'] = tree_census_geoms['tree_count']/tree_census_geoms['cb_acreage']

tree_census_geoms = tree_census_geoms[['cb_num','Tree Quality','Tree Density (per Acre)','geometry']]
tree_census_geoms['cb_num'] = tree_census_geoms['cb_num'].astype(np.int64)
tree_census_geoms=tree_census_geoms.set_index('cb_num')
tree_census_geoms.head()


,Tree Quality,Tree Density (per Acre),geometry
cb_num,,,
101,0.764040,2.399876,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
102,0.730602,5.580675,"MULTIPOLYGON (((-74.00915 40.7425, -74.00902 4..."
103,0.764069,4.375522,"MULTIPOLYGON (((-73.98878 40.73397, -73.98757 ..."
104,0.824395,3.903700,"MULTIPOLYGON (((-73.99394 40.77318, -73.9937 4..."
105,0.673560,2.020331,"MULTIPOLYGON (((-73.97301 40.76428, -73.97141 ..."


# Air Quality Data Sanitation

In [21]:
air_quality_df.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,827080,386,Ozone (O3),Mean,ppb,UHF34,104,Pelham - Throgs Neck,Summer 2022,2022-06-01T00:00:00.000,33.3
1,827061,386,Ozone (O3),Mean,ppb,UHF34,405,Ridgewood - Forest Hills,Summer 2022,2022-06-01T00:00:00.000,34.2
2,827067,386,Ozone (O3),Mean,ppb,UHF34,302,Central Harlem - Morningside Heights,Summer 2022,2022-06-01T00:00:00.000,30.9
3,827081,386,Ozone (O3),Mean,ppb,UHF34,103,Fordham - Bronx Pk,Summer 2022,2022-06-01T00:00:00.000,31.7
4,825967,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104,Pelham - Throgs Neck,Summer 2022,2022-06-01T00:00:00.000,12.0


In [22]:
air_quality_df = air_quality_df.dropna(subset=['geo_join_id'])

In [23]:
air_quality_df['geo_join_id'] = air_quality_df['geo_join_id'].astype('int64')

In [24]:
air_quality_df['name'].unique()

array(['Ozone (O3)', 'Nitrogen dioxide (NO2)', 'Fine particles (PM 2.5)',
       'Annual vehicle miles traveled (cars)',
       'Annual vehicle miles traveled (trucks)',
       'Annual vehicle miles traveled',
       'Respiratory hospitalizations due to PM2.5 (age 20+)',
       'Asthma emergency department visits due to PM2.5',
       'Asthma emergency departments visits due to Ozone',
       'Cardiovascular hospitalizations due to PM2.5 (age 40+)',
       'Cardiac and respiratory deaths due to Ozone',
       'Asthma hospitalizations due to Ozone', 'Deaths due to PM2.5',
       'Boiler Emissions- Total PM2.5 Emissions',
       'Boiler Emissions- Total SO2 Emissions',
       'Boiler Emissions- Total NOx Emissions',
       'Outdoor Air Toxics - Formaldehyde',
       'Outdoor Air Toxics - Benzene'], dtype=object)

In [25]:
air_quality_df = air_quality_df[air_quality_df['name'].str.contains('Outdoor Air Toxics') | air_quality_df['name'].str.contains('Boiler Emissions') ]

In [26]:
simplified_tree_df = tree_census_df[['the_geom', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname', 'boro_ct', 'state']]

In [27]:
simplified_tree_df = simplified_tree_df[(simplified_tree_df['zipcode'] >= 10000) & (simplified_tree_df['zipcode']<11500)]
simplified_tree_df = simplified_tree_df.rename(columns={'the_geom': 'geometry', 'oldName2': 'newName2'})

In [28]:
simplified_tree_df.shape

(670351, 8)

In [29]:
simplified_tree_df = simplified_tree_df.drop_duplicates(subset=['cb_num'])

In [30]:
simplified_tree_df.shape

(59, 8)

In [31]:
air_quality_geo_df =  air_quality_df.merge(simplified_tree_df, left_on='geo_join_id', right_on='cb_num')

In [32]:
air_quality_geo_df.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,geometry,zipcode,zip_city,cb_num,borocode,boroname,boro_ct,state
0,179825,641,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,406,Fresh Meadows,2015,2015-01-01T00:00:00.000,0.4,"{'type': 'Point', 'coordinates': [-73.84421521...",11375,Forest Hills,406,4,Queens,4073900,New York
1,179783,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,104,Pelham - Throgs Neck,2015,2015-01-01T00:00:00.000,2.8,"{'type': 'Point', 'coordinates': [-73.98729652...",10019,New York,104,1,Manhattan,1012700,New York
2,179773,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,404,Bayside - Little Neck,2015,2015-01-01T00:00:00.000,0.9,"{'type': 'Point', 'coordinates': [-73.86529991...",11373,Elmhurst,404,4,Queens,4044302,New York
3,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,"{'type': 'Point', 'coordinates': [-73.84344476...",11414,Howard Beach,410,4,Queens,4089200,New York
4,179792,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,206,Borough Park,2015,2015-01-01T00:00:00.000,1.1,"{'type': 'Point', 'coordinates': [-73.89338225...",10457,Bronx,206,2,Bronx,2037504,New York


# Green Streets Data Sanitation

In [77]:
green_streets_df.head()

,ACRES,BOROUGH,COMMISSIONDATE,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,DESCRIPTION,FEATURESTATUS,GISPROPNUM,GSGROUP,...,PRECINCT,SITENAME,STAREA,STLENGTH,SYSTEM,US_CONGRESS,ZIPCODE,MULTIPOLYGON,GSTYPE,SUBCATEGORY
0,0.070279,Q,2003-12-01T00:00:00.000,414,32,Q-14,Greenstreet,Active,QZ756,40,...,100,Greenstreet,3061.3380279541016,411.5281206503942,QZ756,5,11694,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",NaN,NaN
1,0.070278,Q,2003-12-01T00:00:00.000,414,32,Q-14,Greenstreet,Active,QZ755,40,...,100,Greenstreet,3061.2955627441406,411.52800982410065,QZ755,5,11694,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",NaN,NaN
2,0.070279,Q,2003-12-01T00:00:00.000,414,32,Q-14,Greenstreet,Active,QZ754,40,...,100,Greenstreet,3061.340774536133,411.5280887770174,QZ754,5,11694,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",NaN,NaN
3,0.070278,Q,2003-12-01T00:00:00.000,414,32,Q-14,Greenstreet,Active,QZ753,40,...,100,Greenstreet,3061.3104400634766,411.5281213586527,QZ753,5,11694,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",NaN,NaN
4,0.070279,Q,2003-12-01T00:00:00.000,414,32,Q-14,Greenstreet,Active,QZ752,40,...,100,Greenstreet,3061.337860107422,411.5280235862626,QZ752,5,11694,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",NaN,NaN


In [89]:
df = green_streets_df
df.columns = [i.upper() for i in df.columns]
print(df['DESCRIPTION'].isna().sum())
print((df['DESCRIPTION'] == "").sum())

df['ACRES']=df['ACRES'].astype(np.float64)

0
0


In [90]:
#Check for all invalid values in both acres and zipcode columns
df_invalid = df[
    df['ACRES'].isna() | (df['ACRES'] <= 0) | ~df['ACRES'].apply(lambda x: isinstance(x, (int, float)) and not pd.isna(x)) | (df['ACRES'] == '') |
    df['ZIPCODE'].isna() | (df['ZIPCODE'] == 0) | ~df['ZIPCODE'].apply(lambda x: str(x).isdigit()) | (df['ZIPCODE'] == '')
]

print("Rows with invalid Acres or ZIPCODE values:")
print(df_invalid[['ACRES', 'ZIPCODE']])

print(f"\nNumber of rows with invalid Acres or ZIPCODE: {df_invalid.shape[0]}")


Rows with invalid Acres or ZIPCODE values:
         ACRES              ZIPCODE
632   0.034391         11377, 11378
809   0.008332         11432, 11433
961   0.119891         11374, 11375
1072  0.069322         11213, 11233
1148  0.086223         11207, 11208
1379  0.034699         10463, 10471
1800  0.119379         11423, 11427
1823  0.019118         11370, 11371
1841  0.020756         11370, 11371
1842  0.020843         11370, 11371
1855  0.020318         11370, 11371
1856  0.022438         11370, 11371
1899  0.029911                  NaN
1900  0.044952                  NaN
1901  0.036727                  NaN
1902  0.039310                  NaN
1927  0.043888                 Null
1930  0.237340                 Null
1958  0.019172         11370, 11371
1968  1.203357         10025, 10027
1986  0.018848         11370, 11371
1987  0.019955         11370, 11371
2004  0.841732         10026, 10027
2091  0.409322  10003, 10010, 10016
2093  0.744071         11372, 11377
2098  0.125355       

In [91]:
#Drop all invalid values in both columns
df_cleaned = df[
    ~(df['ACRES'].isna() | (df['ACRES'] <= 0) | ~df['ACRES'].apply(lambda x: isinstance(x, (int, float)) and not pd.isna(x)) | (df['ACRES'] == '') |
    df['ZIPCODE'].isna() | (df['ZIPCODE'] == 0) | ~df['ZIPCODE'].apply(lambda x: str(x).isdigit()) | (df['ZIPCODE'] == '')
    )]
df_cleaned.shape

(2722, 26)

In [92]:
#Check for invalid values 
df_check = df_cleaned[
    df_cleaned['ACRES'].isna() | (df_cleaned['ACRES'] <= 0) | ~df_cleaned['ACRES'].apply(lambda x: isinstance(x, (int, float)) and not pd.isna(x)) | (df_cleaned['ACRES'] == '') |
    df_cleaned['ZIPCODE'].isna() | (df_cleaned['ZIPCODE'] == 0) | ~df_cleaned['ZIPCODE'].apply(lambda x: str(x).isdigit()) | (df_cleaned['ZIPCODE'] == '')
]
print(df_check[['ACRES', 'ZIPCODE']])

Empty DataFrame
Columns: [ACRES, ZIPCODE]
Index: []


In [93]:
#NYC zipcodes range from 10001 to 11697 in five boroughs.
nyc_zipcodes = [str(i) for i in range(10001, 11697)]
Greenstreets_filtered = df_cleaned[df_cleaned['ZIPCODE'].isin(nyc_zipcodes)]

Greenstreets_filtered.shape

(2716, 26)

In [94]:
Greenstreets_filtered.to_csv('Filtered_Greenstreets.csv', index=False)

In [97]:
Greenstreets_filtered = Greenstreets_filtered.rename(columns={'ZIPCODE': 'zipcode'})
Greenstreets_filtered['COMMUNITYBOARD'] = Greenstreets_filtered['COMMUNITYBOARD'].astype(np.int64)
Greenstreets_filtered.head(1)

,ACRES,BOROUGH,COMMISSIONDATE,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,DESCRIPTION,FEATURESTATUS,GISPROPNUM,GSGROUP,...,PRECINCT,SITENAME,STAREA,STLENGTH,SYSTEM,US_CONGRESS,zipcode,MULTIPOLYGON,GSTYPE,SUBCATEGORY
0,0.070279,Q,2003-12-01T00:00:00.000,414,32,Q-14,Greenstreet,Active,QZ756,40,...,100,Greenstreet,3061.3380279541016,411.5281206503942,QZ756,5,11694,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",NaN,NaN


In [101]:
Greenstreets_cb = Greenstreets_filtered\
    .groupby('COMMUNITYBOARD')\
    .agg({'ACRES': 'sum','DESCRIPTION':'count'})\
    .rename(columns={
        'ACRES': 'Greenstreets Acres Total', 
        'DESCRIPTION': 'Greenstreets Counts'
    })
Greenstreets_cb.head()


,Greenstreets Acres Total,Greenstreets Counts
COMMUNITYBOARD,,
101,2.170157,18
102,2.874612,42
103,3.978304,64
104,0.162693,27
105,0.747679,45


# Property Valuation Data Sanitation

In [102]:
property_valuation_df.shape

(1100000, 45)

# Combined Analysis

In [109]:
air_quality_cb = air_quality_geo_df.loc[air_quality_geo_df['geo_type_name']=='CD']

air_quality_cb = air_quality_cb.pivot(
    index='geo_join_id', columns='name', values='data_value'
)

#build master dataset
combined = pd.concat((air_quality_cb, tree_census_geoms, Greenstreets_cb),axis=1)
combined.columns = [
    re.sub(r'[^a-zA-Z0-9\s]', ' ', i).replace(" ",'_') for i in combined.columns
]

#drop geometry col 
combined = combined.drop(columns=['geometry'])

#run data on our target variables (two types of emissions, benzene and formaldehyde)
targets = combined.columns[0:2]
predictors = combined.columns[2:] 

#ensure all values in df are floats...
combined = combined.astype(np.float64)

#run regression analysis, measure hypothesis whether or not some variable is a "predictor" of emissions/pollution 
#that is, the reg weight = 0 under the null, weight != 0 otherwise 
alpha,pv_df = 0.05,[]
for t in targets:
    formula = f"{t} ~ {'+'.join(predictors)}"
    print(formula)
    lm = smf.ols(formula = formula, data = combined).fit() 
    pv=pd.DataFrame(lm.pvalues,columns=[t])
    pv_df.append(pv)

    print(f'R2 [TARGET: {t}]: {lm.rsquared}')
   
pv_df = pd.concat(pv_df,axis=1)
pv_df

Outdoor_Air_Toxics___Benzene ~ Tree_Quality+Tree_Density__per_Acre_+Greenstreets_Acres_Total+Greenstreets_Counts
R2 [TARGET: Outdoor_Air_Toxics___Benzene]: 0.45588011091612957
Outdoor_Air_Toxics___Formaldehyde ~ Tree_Quality+Tree_Density__per_Acre_+Greenstreets_Acres_Total+Greenstreets_Counts
R2 [TARGET: Outdoor_Air_Toxics___Formaldehyde]: 0.4389457551556235


,Outdoor_Air_Toxics___Benzene,Outdoor_Air_Toxics___Formaldehyde
Intercept,2.924678e-08,2.016613e-10
Tree_Quality,4.037430e-05,3.538295e-04
Tree_Density__per_Acre_,9.200899e-01,3.796962e-01
Greenstreets_Acres_Total,3.336006e-04,1.011189e-03
Greenstreets_Counts,8.805001e-03,2.573236e-02


In [104]:
pv_df < alpha

,Outdoor_Air_Toxics___Benzene,Outdoor_Air_Toxics___Formaldehyde
Intercept,True,True
Tree_Quality,True,True
Tree_Density__per_Acre_,False,False
Greenstreets_Acres_Total,True,True
Greenstreets_Counts,True,True
